In [5]:
import numpy as np
array = [[1,2,3],[4,5,6]]
np.shape(np.array(array))

(2, 3)

In [15]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

### Preparing data

In [16]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data", # The location of the downlode file
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [17]:
training_data

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [18]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [Batch size, Channel, Height, Width]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [Batch size, Channel, Height, Width]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


### Creating Models

In [19]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [20]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512), # input is 28 * 28 picture, output is 512 
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
# Every nn.Module subclass implements the operations on input data in the forward method.
    def forward(self, x):
        x = self.flatten(x) # flatten 28 * 28 picture to a feature vector
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [21]:
# Just for testing
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits) #* dim =1 because logits is a 1 by 10 tensor
y_pred = pred_probab.argmax(1) #* dim =1 because pred_probab is a 1 by 10 tensor
print(f"Predicted class: {y_pred}")

Predicted class: tensor([7])


### Optimizing the Model Parameters

In [22]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [26]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset) # return the number of training samples
    model.train() #Tell the model you are going to train so the weights will be updated
    for batch, (X, y) in enumerate(dataloader): # iterates the dataloader 
        # X = X.view(X.size(0),-1)
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X) #! NOTE: Here the returned value is a tuple (decoded, encoded) as defined in forward()
        loss = loss_fn(pred[0], X) # Choose the first value pred[0] as decoded value for the purpose of training
                                    #! NOTE: the second argument should be the original value
        # Backpropagation
        optimizer.zero_grad() # to reset the gradients of model parameters.
        loss.backward() # PyTorch deposits the gradients of the loss w.r.t. each parameter.
        optimizer.step() # to adjust the parameters by the gradients collected in the backward pass.

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [24]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset) # return the number of test samples
    num_batches = len(dataloader) # the first dimension is the number of batches.
    model.eval() #Tell the model you are going to test so the weights will not be updated
    test_loss, correct = 0, 0
    with torch.no_grad(): # NOTE: Context-manager that disables gradient calculation.
        for X, y in dataloader:
            X, y = X.to(device), y.to(device) # to is one of the operations(methods) in tensor object
            pred = model(X)
            test_loss += loss_fn(pred[0], X).item() # adding up all the losses
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [27]:
epochs = 1
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
torch.Size([64, 1, 28, 28])
torch.Size([64, 10])
<class 'torch.Tensor'>
loss: 2.157993  [    0/60000]
torch.Size([64, 1, 28, 28])
torch.Size([64, 10])
<class 'torch.Tensor'>
torch.Size([64, 1, 28, 28])
torch.Size([64, 10])
<class 'torch.Tensor'>
torch.Size([64, 1, 28, 28])
torch.Size([64, 10])
<class 'torch.Tensor'>
torch.Size([64, 1, 28, 28])
torch.Size([64, 10])
<class 'torch.Tensor'>
torch.Size([64, 1, 28, 28])
torch.Size([64, 10])
<class 'torch.Tensor'>
torch.Size([64, 1, 28, 28])
torch.Size([64, 10])
<class 'torch.Tensor'>
torch.Size([64, 1, 28, 28])
torch.Size([64, 10])
<class 'torch.Tensor'>
torch.Size([64, 1, 28, 28])
torch.Size([64, 10])
<class 'torch.Tensor'>
torch.Size([64, 1, 28, 28])
torch.Size([64, 10])
<class 'torch.Tensor'>
torch.Size([64, 1, 28, 28])
torch.Size([64, 10])
<class 'torch.Tensor'>
torch.Size([64, 1, 28, 28])
torch.Size([64, 10])
<class 'torch.Tensor'>
torch.Size([64, 1, 28, 28])
torch.Size([64, 10])
<class 'torch.Ten

### Saving Models

In [26]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


### Loading Models

In [28]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

### Make predictions

In [150]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    print(type(torch.tensor([1,y])))
    # print(pred[0])
    # print(pred[0].argmax(0)) # argmax(0) 0 means the dimension, which is the column
    # predicted, actual = classes[pred[0].argmax(0)], classes[y]
    # print(f'Predicted: "{predicted}", Actual: "{actual}"')

<class 'torch.Tensor'>


ValueError: Expected input batch_size (1) to match target batch_size (2).

In [133]:
pred = model(X)
print(pred.shape)
# test_loss = loss_fn(pred, y).item()
loss_fn(pred, y)
correct += (pred.argmax(1) == y).type(torch.float).sum().item()

torch.Size([1, 10])
tensor(0)


In [98]:
# This computes the matrix multiplication between two tensors. y1, y2, y3 will have the same value
tensor = torch.ones(3,2)
print(tensor)
y1 = tensor @ tensor.T
print(y1)

agg = tensor.sum()
agg_item = agg.item()
print(agg_item, type(agg_item))
agg

tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])
tensor([[2., 2., 2.],
        [2., 2., 2.],
        [2., 2., 2.]])
6.0 <class 'float'>


tensor(6.)

In [95]:
tensor = torch.ones(3,1)
t2 = torch.cat([tensor,tensor],dim=0)
t2

tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]])